# Gradient boosted forest model
Author: Roddy Jaques <br>
*NHS Blood and Transplant*
***
## Assessing a gradient boosted forest model

In this notebook a models for the DBD and DCD cohorts are fit using a gradient boosted forest classifier.

First as usual, load in the data and create the training and test datasets...

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
import sklearn.metrics as mets
import time
%matplotlib inline

# Function to show confusion matrix, balanced accuracy and accuracy for a set of actual and predicted labels
def show_metrics(actual,predict):
    """ Prints the confusion matrix, balanced accuracy and accuracy given datasets of actual and predicted labels
    
    Arguments:
        actual - Dataset of actual labels
        predict - Dataset of predicted labels
     """
    cm = mets.confusion_matrix(actual, predict)
    
    print("********* MODEL METRIC REPORT *********\n\nConfusion matrix:\n")

    print("TN  FN\nFP  TP\n") #this is a reminder of what each part of the confusion matrix means e.g. TP = True Positive
    
    # print the confusion matrix
    print(str(int(cm[0,0])) + "    " + str(int(cm[0,1])))
    print(str(int(cm[1,0])) + "    " + str(int(cm[1,1])) + "\n") 

    # classification report for DBD model
    print("Classification report:\n")
    print(mets.classification_report(actual, predict))

    print("Balanced accuracy: " + str(round(mets.balanced_accuracy_score(actual, predict),2)))

    print("Accuracy: " + str(round(mets.accuracy_score(actual, predict),2)))
    
    # Predicted vs actual consent rates
    cons_rate = int(100 * len(actual[actual==2]) / len(actual) )
    print("\nActual consent rate: " + str(cons_rate))
    
    pred_rate = int(100 * len(predict[predict==2]) / len(predict) )
    print("Predicted consent rate: " + str(pred_rate))



In [25]:
#Read in dataset with all rows included
df = pd.read_sas("Data/alldata3.sas7bdat")

#6931 DBD apps
dbd_apps = df[(df["eli_DBD"]==1)&(df["FAMILY_APPROACHED"]==2)]

#6060 DBD apps to match cohort in paper
dbd_apps = dbd_apps[(dbd_apps["eth_grp"]!=5)&(dbd_apps["FORMAL_APR_WHEN"]!=4)&(dbd_apps["donation_mentioned"]!=-1)
                    &(dbd_apps["FAMILY_WITNESS_BSDT"]!=9)&(dbd_apps["GENDER"]!=9)]
     
#9965 DCD apps
dcd_apps = df[(df["eli_DCD"]==1)&(df["FAMILY_APPROACHED"]==2)]

#9405 DCD apps to match cohort in paper
dcd_apps = dcd_apps[(dcd_apps["GENDER"]!=9)&(dcd_apps["cod_neuro"].notna())&(dcd_apps["eth_grp"]!=5)&(dcd_apps["donation_mentioned"]!=-1)&
                    (~dcd_apps["DTC_WD_TRTMENT_PRESENT"].isin([8,9]))]

# Columns used to create DBD model in paper
dbd_cols = ["wish", "FORMAL_APR_WHEN", "donation_mentioned", "app_nature", "eth_grp", "religion_grp", "GENDER", "FAMILY_WITNESS_BSDT", "DTC_PRESENT_BSD_CONV", 
            "acorn_new", "adult","FAMILY_CONSENT"]

dbd_apps[dbd_cols].astype(int)

dbd_model_data = dbd_apps[dbd_cols]
dbd_model_data2 = pd.get_dummies(data=dbd_model_data,columns=dbd_cols[:-1],drop_first=True)

dbd_features = dbd_model_data2.drop("FAMILY_CONSENT",axis=1)
dbd_consents = dbd_model_data2["FAMILY_CONSENT"]

# Columns used to create DCD model in paper
dcd_cols = ["wish", "donation_mentioned", 
            "app_nature", "eth_grp", "religion_grp", "GENDER", "DTC_WD_TRTMENT_PRESENT", 
            "acorn_new", "adult","cod_neuro","FAMILY_CONSENT"]

dcd_apps[dbd_cols].astype(int)

dcd_model_data = dcd_apps[dcd_cols]
dcd_model_data2 = pd.get_dummies(data=dcd_model_data,columns=dcd_cols[:-1],drop_first=True)

dcd_features = dcd_model_data2.drop("FAMILY_CONSENT",axis=1)
dcd_consents = dcd_model_data2["FAMILY_CONSENT"]

# creating a train and testing dataset for DBD and DCD approaches

# 1382 consents, 618 non-consents in test data
DBD_X_train, DBD_X_test, DBD_y_train, DBD_y_test = train_test_split(dbd_features,dbd_consents, test_size=0.33, random_state=10)

# 1865 consents, 1239 non-consents in t
DCD_X_train, DCD_X_test, DCD_y_train, DCD_y_test = train_test_split(dcd_features,dcd_consents, test_size=0.33, random_state=10)

/Users/roddy.jacques/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sas/sas7bdat.py:800: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  rslt[name] = self._byte_chunk[jb, :].view(dtype=self.byte_order + "d")
/Users/roddy.jacques/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sas/sas7bdat.py:809: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  rslt[name] = self._string_chunk[js, :]


<br>
Next, fit a Gradient boosted forest classifier to the DBD and DCD data with default hyperparameters.
<br><br>

In [29]:
# fitting tree to training data 
boost_model = GradientBoostingClassifier()

# fit to DBD training data 
DBD_boost = boost_model.fit(DBD_X_train,DBD_y_train)

# predict and evaluate test data
DBD_preds = DBD_boost.predict(DBD_X_test)

In [26]:
# show metrics for DBD model
show_metrics(DBD_y_test, DBD_preds)

********* MODEL METRIC REPORT *********

Confusion matrix:

TN  FN
FP  TP

234    384
104    1278

Classification report:

              precision    recall  f1-score   support

         1.0       0.69      0.38      0.49       618
         2.0       0.77      0.92      0.84      1382

    accuracy                           0.76      2000
   macro avg       0.73      0.65      0.66      2000
weighted avg       0.75      0.76      0.73      2000

Balanced accuracy: 0.65
Accuracy: 0.76

Actual consent rate: 69
Predicted consent rate: 83


In [27]:
# fit to DCD training data
DCD_boost = boost_model.fit(DCD_X_train,DCD_y_train)

# fit and evalute DCD test data
DCD_preds = DCD_boost.predict(DCD_X_test)

In [28]:
# print metrics for DCD model
show_metrics(DCD_y_test, DCD_preds)

********* MODEL METRIC REPORT *********

Confusion matrix:

TN  FN
FP  TP

790    449
388    1477

Classification report:

              precision    recall  f1-score   support

         1.0       0.67      0.64      0.65      1239
         2.0       0.77      0.79      0.78      1865

    accuracy                           0.73      3104
   macro avg       0.72      0.71      0.72      3104
weighted avg       0.73      0.73      0.73      3104

Balanced accuracy: 0.71
Accuracy: 0.73

Actual consent rate: 60
Predicted consent rate: 62


***
#### DBD model
The DBD gradient boosted model has very low recall for the non-consents, worse than the logistic regression model (non-consent recall=0.44). And the balanced accuracy is lower than the logistic regression model's (BA = 0.67). 

The random forest model, even untuned, has outperforms the boosted model, which is dissapointing given the reputation of boosted forest models. 

#### DCD model
The DCD model here does slightly better than the logisitc regression, with a non-consent recall of 0.64, 0.02 higher then logitistic regression. This does perform better than the untuned random forest, on recall for both classes and overall balanced accuracy and accuracy. 


***

## Hyperparameter tuning

Tuning the hpyerparameters to try and improve the performance of the models. I'm using a cross validated gridsearch, optimising on balanced accuracy so that the balanced accuracy score isn't inflated by a high recall in the consent class.<br>
The hyper parameters and range to be explored are: <br>
* max_depth - the maximum tree depth. From 1 to 200 in increments of 25.
* n_estimators - the number of boosting stages. From 40 to 300 in increments of 30.

In [17]:
# use a 5 fold cross validated grid search to find optimal hyperparameters
cv_boost_model = GradientBoostingClassifier(random_state=66)

# hyperparameters to test
params = {'max_depth':np.arange(1,200,step=25),'n_estimators':np.arange(40,300,step=30),'learning_rate':np.arange(0.05,0.6,step=0.1)}

start_time = time.time()

# train model for highest balanced accuracy
dbd_gs_boost_model = GridSearchCV(cv_boost_model, param_grid=params, scoring="balanced_accuracy",cv=5,n_jobs=3)

dbd_gs_boost_model.fit(DBD_X_train,DBD_y_train)

runtime = time.time() - start_time
print("Runtime = {}minutes".format(round(runtime/60,1)))

# balanced accuracy of best model
dbd_gs_boost_model.score(DBD_X_train,DBD_y_train)

#
print(dbd_gs_boost_model.best_params_)
print(dbd_gs_boost_model.best_score_)

Fitting 5 folds for each of 432 candidates, totalling 2160 fits
[CV 1/5] END learning_rate=0.05, max_depth=1, n_estimators=40;, score=0.544 total time=   0.1s
[CV 1/5] END learning_rate=0.05, max_depth=1, n_estimators=70;, score=0.544 total time=   0.1s
[CV 4/5] END learning_rate=0.05, max_depth=1, n_estimators=70;, score=0.542 total time=   0.1s
[CV 2/5] END learning_rate=0.05, max_depth=1, n_estimators=100;, score=0.589 total time=   0.1s
[CV 5/5] END learning_rate=0.05, max_depth=1, n_estimators=100;, score=0.579 total time=   0.1s
[CV 3/5] END learning_rate=0.05, max_depth=1, n_estimators=130;, score=0.589 total time=   0.1s
[CV 1/5] END learning_rate=0.05, max_depth=1, n_estimators=160;, score=0.573 total time=   0.1s
[CV 4/5] END learning_rate=0.05, max_depth=1, n_estimators=160;, score=0.575 total time=   0.1s
[CV 2/5] END learning_rate=0.05, max_depth=1, n_estimators=190;, score=0.615 total time=   0.2s
[CV 5/5] END learning_rate=0.05, max_depth=1, n_estimators=190;, score=0.60

In [19]:
# predict DBD test data consent
DBD_preds = dbd_gs_boost_model.predict(DBD_X_test)

# print confusion matrix and other measures
cm = mets.confusion_matrix(DBD_y_test, DBD_preds)

print("TN  FN\nFP  TP\n")
print(str(int(cm[0,0])) + "    " + str(int(cm[0,1])))
print(str(int(cm[1,0])) + "    " + str(int(cm[1,1])) + "\n") 

# classification report for DBD model
print(mets.classification_report(DBD_y_test, DBD_preds))

print("Balanced accuracy: " + str(round(mets.balanced_accuracy_score(DBD_y_test, DBD_preds),2)))

print("Accuracy: " + str(round(mets.accuracy_score(DBD_y_test, DBD_preds),2)))

TN  FN
FP  TP

366    252
369    1013

              precision    recall  f1-score   support

         1.0       0.50      0.59      0.54       618
         2.0       0.80      0.73      0.77      1382

    accuracy                           0.69      2000
   macro avg       0.65      0.66      0.65      2000
weighted avg       0.71      0.69      0.70      2000

Balanced accuracy: 0.66
Accuracy: 0.69


In [ ]:
start_time = time.time()

dcd_gs_boost_model = GridSearchCV(cv_boost_model, param_grid=params, scoring="balanced_accuracy",cv=5,n_jobs=3)

dcd_gs_boost_model.fit(DCD_X_train,DCD_y_train)

runtime = time.time() - start_time
print("Runtime = {}minutes".format(round(runtime/60,1)))

dcd_gs_boost_model.score(DCD_X_train,DCD_y_train)

print(dcd_gs_boost_model.best_params_)
print(dcd_gs_boost_model.best_score_)

In [ ]:
DCD_preds = dcd_gs_boost_model.predict(DCD_X_test)

cm = mets.confusion_matrix(DCD_y_test, DCD_preds)

print("TN  FN\nFP  TP\n")
print(str(int(cm[0,0])) + "    " + str(int(cm[0,1])))
print(str(int(cm[1,0])) + "    " + str(int(cm[1,1])) + "\n") 

# classification report for DCD model
print(mets.classification_report(DCD_y_test, DCD_preds))

print("Balanced accuracy: " + str(round(mets.balanced_accuracy_score(DCD_y_test, DCD_preds),2)))

print("Accuracy: " + str(round(mets.accuracy_score(DCD_y_test, DCD_preds),2)))